# Script-Languages-Container UI

<b>This notebook is not supposed to be used on its own.<b>


In [1]:
from exasol.nb_connector.utils import upward_file_search

# This NB may be running from various locations in the NB hierarchy.
# Need to search for the styles NB from the current directory upwards.

%run {upward_file_search('utils/ui_styles.ipynb')}
%run {upward_file_search('utils/popup_message_ui.ipynb')}

Output()

In [6]:
slc_source_store_key = "slc_source"
slc_source_slc_repo_option_value = "Clone script languages release repository"
slc_target_dir_store_key = "slc_target_dir"
slc_source_flavor_store_key = "slc_flavor"

from ipyfilechooser import FileChooser

NOTEBOOKS_DIRECTORY = "/home/jupyter/notebooks"
def clone_slc_repo(conf: Secrets) -> bool:
    return conf.get(slc_source_store_key) == slc_source_slc_repo_option_value

def get_slc_source_selection_ui(conf: Secrets) -> widgets.Widget:
    """
    Creates a UI form for choosing the flavor source
    """

    ui_look = get_config_styles()
    ui_look.input_layout.max_width = "500px"
    
    slc_sources = [slc_source_slc_repo_option_value, "Use existing repository"]
    default_value = conf.get(slc_source_store_key)

    flavor_source_selector = widgets.RadioButtons(options=slc_sources, value=default_value, 
                                                  layout=ui_look.input_layout, style=ui_look.input_style)
    select_btn = widgets.Button(description='Select', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Flavor choice', style=ui_look.header_style, layout=ui_look.header_layout)


    def select_flavor_source(btn):
        conf.save(slc_source_store_key, flavor_source_selector.value)
        btn.icon = 'check'

    def on_value_change(change):
        select_btn.icon = 'pen'

    select_btn.on_click(select_flavor_source)
    flavor_source_selector.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([flavor_source_selector], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), select_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui

def get_slc_target_dir_ui(conf: Secrets) -> widgets.Widget:
    """
    Creates a UI form for editing the Script-Languages-Container repository configuration.
    """
    default_target_dir = NOTEBOOKS_DIRECTORY
    target_dir_chooser_widget = FileChooser(path=default_target_dir, select_default=True)
    target_dir_chooser_widget.show_only_dirs = True
    target_dir_chooser_widget.sandbox_path = NOTEBOOKS_DIRECTORY
    
    inputs = [
        [
            ('Target Base Directory', target_dir_chooser_widget, slc_target_dir_store_key),
        ]
    ]

    ui_look = get_config_styles()
    ui_look.row_layout.max_width = "500px"
    ui_look.group_layout.max_width = "500px"
    save_btn = widgets.Button(description='Save', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Target Directory', style=ui_look.header_style, layout=ui_look.header_layout)

    def save_configuration(btn):
        target_dir = Path(target_dir_chooser_widget.selected) / "script_languages_release"
        conf.save(slc_target_dir_store_key, str(target_dir))
        btn.icon = 'check'

    def on_value_change(change):
        save_btn.icon = 'pen'

    save_btn.on_click(save_configuration)

    # Apply the styles and layouts to the input fields
    target_dir_chooser_widget.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([target_dir_chooser_widget], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), save_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    

    if clone_slc_repo(conf):
        return ui

def get_existing_slc_ui(conf: Secrets) -> widgets.Widget:
    """
    Creates a UI form for choosing the existing script-languages repository.
    """
    default_src_dir = conf.get(slc_target_dir_store_key, '')
    select_default = True if slc_target_dir_store_key else False
    src_dir_chooser_widget = FileChooser(path=default_src_dir, select_default=select_default)
    src_dir_chooser_widget.show_only_dirs = True
    src_dir_chooser_widget.sandbox_path = NOTEBOOKS_DIRECTORY
    
    inputs = [
        [
            ('Flavor source directory', src_dir_chooser_widget, slc_target_dir_store_key),
        ]
    ]

    ui_look = get_config_styles()
    ui_look.row_layout.max_width = "500px"
    ui_look.group_layout.max_width = "500px"
    save_btn = widgets.Button(description='Save', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Existing script-languages directory', style=ui_look.header_style, layout=ui_look.header_layout)


    def save_configuration(btn):
        target_dir = Path(src_dir_chooser_widget.selected)
        if not (target_dir / "flavors").is_dir():
            popup_message("Invalid directory. You need to choose a valid script-languages repository.")
        conf.save(slc_target_dir_store_key, str(src_dir_chooser_widget.selected))
        btn.icon = 'check'

    def on_value_change(change):
        save_btn.icon = 'pen'

    save_btn.on_click(save_configuration)

    # Apply the styles and layouts to the input fields
    src_dir_chooser_widget.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([src_dir_chooser_widget], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), save_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    

    if not clone_slc_repo(conf):
        return ui


def get_flavor_selection_ui(conf: Secrets) -> widgets.Widget:
    """
    Creates a UI form for choosing from a list of flavors.
    """

    from os import listdir
    from os.path import isdir
    from pathlib import Path

    slc_dir = conf.get(slc_target_dir_store_key)
    


    flavor_base_dir = Path(slc_dir) / "flavors"
    flavor_dirs =  [flavor_dir.name for flavor_dir in flavor_base_dir.iterdir() if flavor_dir.is_dir()]

    ui_look = get_config_styles()
    ui_look.input_layout.max_width = "500px"

    #Setup default value
    #First we check if we can match the value from the Secret store
    default_src_dir = conf.get(slc_source_flavor_store_key, '')
    default_src_dir_value = ''
    if default_src_dir:
        matching_flavor_dirs = [flavor_dir.name for flavor_dir in flavor_base_dir.iterdir() if flavor_dir.is_dir() and str(flavor_dir) == default_src_dir]
        if len(matching_flavor_dirs) == 1:
            default_src_dir_value = matching_flavor_dirs[0]
    #If not found in the secret store, we choose the first path which matches the template-
    if not default_src_dir_value:
        default_src_dir_value = "template-Exasol-all-python-3.10"
    flavor_selector = widgets.RadioButtons(options=flavor_dirs, value=default_src_dir_value, 
                                           layout=ui_look.input_layout, style=ui_look.input_style)
    select_btn = widgets.Button(description='Select', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Flavor choice', style=ui_look.header_style, layout=ui_look.header_layout)


    def select_flavor(btn):
        conf.save(slc_source_flavor_store_key, str(flavor_selector.value))
        btn.icon = 'check'

    def on_value_change(change):
        select_btn.icon = 'pen'

    select_btn.on_click(select_flavor)
    flavor_selector.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([flavor_selector], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), select_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui

NameError: name 'Secrets' is not defined